In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import os, tarfile
import re
import pandas as pd
import numpy as np

In [24]:
kmer_data_path = "/data/projects/Enhancer/New_Finetune_data_with_seqFeature-encodeFeature_raw_data/CPG2_greater/random_12K/DNABERT_finetune_12kSampled_data_Dataset2.csv"
encode_data_path = "/data/projects/Enhancer/New_Finetune_data_with_seqFeature-encodeFeature_raw_data/CPG2_greater/random_12K/Dataset2_encode-features_12kSampled_data.csv"
sequence_data_path = "/data/projects/Enhancer/New_Finetune_data_with_seqFeature-encodeFeature_raw_data/CPG2_greater/random_12K/Dataset2_sequence-features_12kSampled_data.csv"
encode_sequence_data_path = "/data/projects/Enhancer/New_Finetune_data_with_seqFeature-encodeFeature_raw_data/CPG2_greater/random_12K/Dataset2_sequence-encode-features_12kSampled_data.csv"

In [25]:
# Use a fixed random state for reproducibility
random_state = 42

In [26]:
df_kmer = pd.read_csv(kmer_data_path , sep=",")
df_encode = pd.read_csv(encode_data_path , sep=",")
df_sequence = pd.read_csv(sequence_data_path , sep=",")
df_encode_sequence = pd.read_csv(encode_sequence_data_path, sep=",")

In [27]:
print(df_encode.shape, df_kmer.shape, df_sequence.shape, df_encode_sequence.shape)

(24000, 4001) (24000, 2) (24000, 11) (24000, 4011)


In [28]:
df_encode = df_encode.drop(df_encode.columns[0], axis=1)
df_encode

,3xFLAG-AHR-HepG2_0,3xFLAG-ARID4B-HepG2_0,3xFLAG-ATF1-HepG2_0,3xFLAG-ATF1-K562_0,3xFLAG-BCL6-HepG2_0,3xFLAG-CEBPA-HepG2_0,3xFLAG-CEBPG-HepG2_0,3xFLAG-CREB1-HepG2_0,3xFLAG-DMAP1-HepG2_0,3xFLAG-DNMT3B-HepG2_0,...,eGFP-ZSCAN16-HEK293_0,eGFP-ZSCAN18-HEK293_0,eGFP-ZSCAN21-HEK293_0,eGFP-ZSCAN23-HEK293_0,eGFP-ZSCAN26-HEK293_0,eGFP-ZSCAN30-HEK293_0,eGFP-ZSCAN4-HEK293_0,eGFP-ZSCAN5A-HEK293_0,eGFP-ZSCAN5C-HEK293_0,eGFP-ZXDB-HEK293_0
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
df_sequence = df_sequence.drop(df_sequence.columns[0], axis=1)
df_sequence

,A_Fraction,C_Fraction,G_Fraction,T_Fraction,PurPyr_Fraction,AmKe_Fraction,WeSt_Fraction,CpG1,CpG2,CpG3
0,0.290,0.235,0.290,0.185,0.16,0.05,-0.05,0.201005,0.363636,0.520202
1,0.210,0.300,0.190,0.300,-0.20,0.02,0.02,0.150754,0.257576,0.358586
2,0.165,0.320,0.225,0.290,-0.22,-0.03,-0.09,0.180905,0.308081,0.358586
3,0.175,0.305,0.305,0.215,-0.04,-0.04,-0.22,0.231156,0.439394,0.601010
4,0.175,0.290,0.250,0.285,-0.15,-0.07,-0.08,0.130653,0.282828,0.484848
...,...,...,...,...,...,...,...,...,...,...
23995,0.310,0.245,0.270,0.175,0.16,0.11,-0.03,0.201005,0.348485,0.383838
23996,0.165,0.225,0.325,0.285,-0.02,-0.22,-0.10,0.150754,0.287879,0.459596
23997,0.265,0.230,0.240,0.265,0.01,-0.01,0.06,0.160804,0.257576,0.393939
23998,0.240,0.225,0.325,0.210,0.13,-0.07,-0.10,0.170854,0.272727,0.313131


In [30]:
df_encode_sequence = df_encode_sequence.drop(df_encode_sequence.columns[0], axis=1)
df_encode_sequence

,A_Fraction,C_Fraction,G_Fraction,T_Fraction,PurPyr_Fraction,AmKe_Fraction,WeSt_Fraction,CpG1,CpG2,CpG3,...,eGFP-ZSCAN16-HEK293_0,eGFP-ZSCAN18-HEK293_0,eGFP-ZSCAN21-HEK293_0,eGFP-ZSCAN23-HEK293_0,eGFP-ZSCAN26-HEK293_0,eGFP-ZSCAN30-HEK293_0,eGFP-ZSCAN4-HEK293_0,eGFP-ZSCAN5A-HEK293_0,eGFP-ZSCAN5C-HEK293_0,eGFP-ZXDB-HEK293_0
0,0.290,0.235,0.290,0.185,0.16,0.05,-0.05,0.201005,0.363636,0.520202,...,0,0,0,0,0,0,0,0,0,0
1,0.210,0.300,0.190,0.300,-0.20,0.02,0.02,0.150754,0.257576,0.358586,...,0,0,0,0,0,0,0,0,0,0
2,0.165,0.320,0.225,0.290,-0.22,-0.03,-0.09,0.180905,0.308081,0.358586,...,0,0,0,0,0,0,0,0,0,0
3,0.175,0.305,0.305,0.215,-0.04,-0.04,-0.22,0.231156,0.439394,0.601010,...,0,0,0,0,0,0,0,0,0,0
4,0.175,0.290,0.250,0.285,-0.15,-0.07,-0.08,0.130653,0.282828,0.484848,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23995,0.310,0.245,0.270,0.175,0.16,0.11,-0.03,0.201005,0.348485,0.383838,...,0,0,0,0,0,0,0,0,0,0
23996,0.165,0.225,0.325,0.285,-0.02,-0.22,-0.10,0.150754,0.287879,0.459596,...,0,0,0,0,0,0,0,0,0,0
23997,0.265,0.230,0.240,0.265,0.01,-0.01,0.06,0.160804,0.257576,0.393939,...,0,0,0,0,0,0,0,0,0,0
23998,0.240,0.225,0.325,0.210,0.13,-0.07,-0.10,0.170854,0.272727,0.313131,...,0,0,0,0,0,0,0,0,0,0


In [31]:
output_folder_path = "/data/projects/Enhancer/Finetuned_data/CPG2_greater_random12K"

In [32]:
output_folder_path

'/data/projects/Enhancer/Finetuned_data/CPG2_greater_random12K'

In [33]:
if not os.path.exists(output_folder_path + '/Evaluation_data'):
    os.makedirs(output_folder_path + '/Evaluation_data')

In [13]:
df.head()

NameError: name 'df' is not defined

In [34]:
# Shuffle and split datasets
train_1, test_1 = train_test_split(df_kmer, test_size=0.2, shuffle=True, random_state=random_state)
train_2, test_2 = train_test_split(df_encode, test_size=0.2, shuffle=True, random_state=random_state)
train_3, test_3 = train_test_split(df_sequence, test_size=0.2, shuffle=True, random_state=random_state)
train_4, test_4 = train_test_split(df_encode_sequence, test_size=0.2, shuffle=True, random_state=random_state)

In [35]:
test_1, val_1 = train_test_split(test_1, test_size=0.5, shuffle=True, random_state=random_state)
test_2, val_2 = train_test_split(test_2, test_size=0.5, shuffle=True, random_state=random_state)
test_3, val_3 = train_test_split(test_3, test_size=0.5, shuffle=True, random_state=random_state)
test_4, val_4 = train_test_split(test_4, test_size=0.5, shuffle=True, random_state=random_state)

In [36]:
train_1

,Sequence,Label
3572,CTCCCT TCCCTC CCCTCC CCTCCC CTCCCT TCCCTT CCCT...,1
16782,TAGGTT AGGTTG GGTTGG GTTGGG TTGGGT TGGGTG GGGT...,0
20543,TGGGAG GGGAGC GGAGCC GAGCCC AGCCCG GCCCGA CCCG...,0
12368,GGGATT GGATTA GATTAC ATTACA TTACAG TACAGG ACAG...,0
19955,TCATGT CATGTG ATGTGC TGTGCT GTGCTT TGCTTC GCTT...,0
...,...,...
21575,TCTTAC CTTACA TTACAG TACAGA ACAGAC CAGACT AGAC...,0
5390,GGGTTT GGTTTC GTTTCA TTTCAC TTCACC TCACCA CACC...,1
860,TCAGGA CAGGAG AGGAGT GGAGTT GAGTTC AGTTCA GTTC...,1
15795,CAAAAT AAAATG AAATGC AATGCA ATGCAT TGCATC GCAT...,0


In [37]:
train_2

,3xFLAG-AHR-HepG2_0,3xFLAG-ARID4B-HepG2_0,3xFLAG-ATF1-HepG2_0,3xFLAG-ATF1-K562_0,3xFLAG-BCL6-HepG2_0,3xFLAG-CEBPA-HepG2_0,3xFLAG-CEBPG-HepG2_0,3xFLAG-CREB1-HepG2_0,3xFLAG-DMAP1-HepG2_0,3xFLAG-DNMT3B-HepG2_0,...,eGFP-ZSCAN16-HEK293_0,eGFP-ZSCAN18-HEK293_0,eGFP-ZSCAN21-HEK293_0,eGFP-ZSCAN23-HEK293_0,eGFP-ZSCAN26-HEK293_0,eGFP-ZSCAN30-HEK293_0,eGFP-ZSCAN4-HEK293_0,eGFP-ZSCAN5A-HEK293_0,eGFP-ZSCAN5C-HEK293_0,eGFP-ZXDB-HEK293_0
3572,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16782,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20543,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12368,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19955,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21575,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5390,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
860,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15795,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [38]:
train_3

,A_Fraction,C_Fraction,G_Fraction,T_Fraction,PurPyr_Fraction,AmKe_Fraction,WeSt_Fraction,CpG1,CpG2,CpG3
3572,0.175,0.265,0.220,0.340,-0.21,-0.12,0.03,0.150754,0.267677,0.353535
16782,0.280,0.240,0.265,0.215,0.09,0.04,-0.01,0.140704,0.257576,0.343434
20543,0.315,0.210,0.300,0.175,0.23,0.05,-0.02,0.180905,0.262626,0.262626
12368,0.220,0.245,0.240,0.295,-0.08,-0.07,0.03,0.150754,0.287879,0.373737
19955,0.215,0.315,0.245,0.225,-0.08,0.06,-0.12,0.190955,0.333333,0.429293
...,...,...,...,...,...,...,...,...,...,...
21575,0.305,0.245,0.200,0.250,0.01,0.10,0.11,0.130653,0.252525,0.414141
5390,0.270,0.240,0.215,0.275,-0.03,0.02,0.09,0.160804,0.262626,0.292929
860,0.280,0.225,0.300,0.195,0.16,0.01,-0.05,0.221106,0.303030,0.328283
15795,0.300,0.215,0.280,0.205,0.16,0.03,0.01,0.201005,0.343434,0.358586


In [39]:
train_4

,A_Fraction,C_Fraction,G_Fraction,T_Fraction,PurPyr_Fraction,AmKe_Fraction,WeSt_Fraction,CpG1,CpG2,CpG3,...,eGFP-ZSCAN16-HEK293_0,eGFP-ZSCAN18-HEK293_0,eGFP-ZSCAN21-HEK293_0,eGFP-ZSCAN23-HEK293_0,eGFP-ZSCAN26-HEK293_0,eGFP-ZSCAN30-HEK293_0,eGFP-ZSCAN4-HEK293_0,eGFP-ZSCAN5A-HEK293_0,eGFP-ZSCAN5C-HEK293_0,eGFP-ZXDB-HEK293_0
3572,0.175,0.265,0.220,0.340,-0.21,-0.12,0.03,0.150754,0.267677,0.353535,...,0,0,0,0,0,0,0,0,0,0
16782,0.280,0.240,0.265,0.215,0.09,0.04,-0.01,0.140704,0.257576,0.343434,...,0,0,0,0,0,0,0,0,0,0
20543,0.315,0.210,0.300,0.175,0.23,0.05,-0.02,0.180905,0.262626,0.262626,...,0,0,0,0,0,0,0,0,0,0
12368,0.220,0.245,0.240,0.295,-0.08,-0.07,0.03,0.150754,0.287879,0.373737,...,0,0,0,0,0,0,0,0,0,0
19955,0.215,0.315,0.245,0.225,-0.08,0.06,-0.12,0.190955,0.333333,0.429293,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21575,0.305,0.245,0.200,0.250,0.01,0.10,0.11,0.130653,0.252525,0.414141,...,0,0,0,0,0,0,0,0,0,0
5390,0.270,0.240,0.215,0.275,-0.03,0.02,0.09,0.160804,0.262626,0.292929,...,0,0,0,0,0,0,0,0,0,0
860,0.280,0.225,0.300,0.195,0.16,0.01,-0.05,0.221106,0.303030,0.328283,...,0,0,0,0,0,0,0,0,0,0
15795,0.300,0.215,0.280,0.205,0.16,0.03,0.01,0.201005,0.343434,0.358586,...,0,0,0,0,0,0,0,0,0,0


In [40]:
train_1.to_csv(output_folder_path+"/train.tsv", sep="\t", index=False)
test_1.to_csv(output_folder_path+"/dev.tsv", sep="\t", index=False)
val_1.to_csv(output_folder_path+"/Evaluation_data/dev.tsv", sep="\t", index=False)

In [41]:
train_2.to_csv(output_folder_path+"/encode_train.tsv", sep="\t", index=False)
test_2.to_csv(output_folder_path+"/encode_dev.tsv", sep="\t", index=False)
val_2.to_csv(output_folder_path+"/Evaluation_data/encode_dev.tsv", sep="\t", index=False)

In [42]:
train_3.to_csv(output_folder_path+"/sequence_train.tsv", sep="\t", index=False)
test_3.to_csv(output_folder_path+"/sequence_dev.tsv", sep="\t", index=False)
val_3.to_csv(output_folder_path+"/Evaluation_data/sequence_dev.tsv", sep="\t", index=False)

In [43]:
train_4.to_csv(output_folder_path+"/encode_sequence_train.tsv", sep="\t", index=False)
test_4.to_csv(output_folder_path+"/encode_sequence_dev.tsv", sep="\t", index=False)
val_4.to_csv(output_folder_path+"/Evaluation_data/encode_sequence_dev.tsv", sep="\t", index=False)